<a href="https://colab.research.google.com/github/BongYoonCHOI/MRC_Project/blob/main/MRC_Mini_Project(2020517014%2C_%EC%B5%9C%EB%B4%89%EC%9C%A4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MRC Mini Project (2020517014, 최봉윤)

본 실습 자료는 **DFC615(00) 인공지능과자연언어처리기술 강의 실습**을 위해 **고려대학교 자연어처리연구실 (NLP & AI Lab)**에서 제작했습니다.

☠️ 학습 이외의 목적으로 무단 배포를 금지합니다. ☠️



```
version 1.0 (2021.06.03)
created by: 박성진, 임희석 (고려대학교 자연어처리 연구실)
email: ec_park@korea.ac.kr
```



### **구글 드라이브 연동** 😸

In [4]:
# https://colab.research.google.com/drive/1EebE_t1s3hS6_oAvPH5DiOsLq3WlTTRZ?usp=sharing#scrollTo=OLrvpcBDTP97
import os
from google.colab import drive
drive.mount('/content/gdrive/') 
os.chdir('/content/gdrive/My Drive/Colab Notebooks/DFC615') ## 현재 작업 환경으로 설정한 경로를 입력하세요

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


# 1. MRC on [Korqard](https://korquad.github.io/KorQuad%201.0/) with KoElectra


## 1.1 Dataset



#### **한국어 기계 독해 기반의 질의 응답 데이터셋**

```
KorQuAD 1.0: https://korquad.github.io/KorQuad%201.0/

KorQuAD 2.0: https://korquad.github.io/
```

```
KorQuAD는 한국어 기반의 기계 독해를 위해 제작한 데이터셋이며, 데이터셋은 지문과 질의 답변으로 이루어져 있습니다.
```





In [5]:
import os
import json
os.chdir('/content/gdrive/My Drive/Colab Notebooks/DFC615')
gdrive_path = "/content/gdrive/My Drive/Colab Notebooks/DFC615"


In [6]:
output_dir = gdrive_path
train_file = os.path.join(gdrive_path, "KorQuAD_v1.0_train.json")
dev_file = os.path.join(gdrive_path, "KorQuAD_v1.0_dev.json")

korquad_train = json.load(open(train_file,'r',encoding='utf-8'))    
korquad_dev = json.load(open(dev_file,'r',encoding='utf-8'))    

In [7]:
train_cnt, val_cnt = 0, 0
for idx, train in enumerate(korquad_train['data']):
    title_name = train['title']
    for paragraph in train['paragraphs']:
        context=paragraph['context'] 
        for qas in paragraph['qas']:
            train_cnt += 1

for idx, train in enumerate(korquad_dev['data']): 
    title_name = train['title']
    for paragraph in train['paragraphs']:
        context=paragraph['context'] 
        for qas in paragraph['qas']:
            val_cnt += 1
print(f'Train: {train_cnt} | Val: {val_cnt}')


Train: 60407 | Val: 5774


In [10]:
!pip install transformers==3.3.1
!pip install attrdict

     |████████████████████████████████| 1.1MB 27.0MB/s 
     |████████████████████████████████| 901kB 46.1MB/s 
     |████████████████████████████████| 3.0MB 22.6MB/s 
     |████████████████████████████████| 1.2MB 50.2MB/s 


## 2.1 필요한 라이브러리

In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from collections import Counter
import sys
import argparse
import string
import re
import glob
import logging
import random 
import timeit

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from fastprogress.fastprogress import master_bar, progress_bar
from attrdict import AttrDict

from transformers import (
    AdamW,
    get_linear_schedule_with_warmup,
    squad_convert_examples_to_features
)
from transformers.data.metrics.squad_metrics import (
    compute_predictions_logits,
    squad_evaluate,
)

from transformers import (
    ElectraForQuestionAnswering,
    XLMRobertaForQuestionAnswering,
    
    ElectraTokenizer,
    XLMRobertaTokenizer,

    ElectraConfig,
    XLMRobertaConfig
)
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor
from transformers import pipeline
logger = logging.getLogger(__name__)

## 2.2 Fine-Tuning을 위한 코드

In [12]:
def init_logger():
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        
        level=logging.INFO,
    )

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if not args.no_cuda and torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)

MODEL_FOR_QUESTION_ANSWERING = {
    "koelectra-base-v3": ElectraForQuestionAnswering,
    "koelectra-small-v3": ElectraForQuestionAnswering,
}
TOKENIZER_CLASSES = {
    "koelectra-base-v3": ElectraTokenizer,
    "koelectra-small-v3": ElectraTokenizer,
}
CONFIG_CLASSES = {
    "koelectra-base-v3": ElectraConfig,
    "koelectra-small-v3": ElectraConfig,
}

In [52]:
'''KorQuAD v1.0에 대한 공식 평가 스크립트 '''
'''본 스크립트는 SQuAD v1.1 평가 스크립트 https://rajpurkar.github.io/SQuAD-explorer/ 를 바탕으로 작성됨.'''

def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 및 조사 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        text = text.rstrip("을")
        text = text.rstrip("를")
        text = text.rstrip("의")
        text = text.rstrip("에서")
        text = text.rstrip("에게")
        text = text.rstrip("에겐")
        text = text.rstrip("에도")
        text = text.rstrip("에")
        text = re.sub("에", " ", text)
        text = text.rstrip("도")
        text = text.rstrip("은")
        text = text.rstrip("는")
        text = text.rstrip("와")
        text = text.rstrip("과")
        text = text.rstrip("이가")
        text = text.rstrip("이라")
        text = text.rstrip("이후")
        text = text.rstrip("이")
        text = text.rstrip("으로")
        text = text.rstrip("로서")
        text = re.sub("로서", " ", text)
        text = text.rstrip("로")
        text = text.rstrip("라면서")
        text = text.rstrip("까지만")
        
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # F1 by character
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(dataset, predictions):
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    message = 'Unanswered question ' + qa['id'] + \
                              ' will receive score 0.'
                    print(message, file=sys.stderr)
                    continue
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    return {'official_exact_match': exact_match, 'official_f1': f1}


def eval_during_train(args, step):
    expected_version = 'KorQuAD_v1.0'

    dataset_file = os.path.join(args.data_dir, args.predict_file)
    prediction_file = os.path.join(args.output_dir, 'predictions_{}.json'.format(step))

    with open(dataset_file) as dataset_f:
        dataset_json = json.load(dataset_f)
        read_version = "_".join(dataset_json['version'].split("_")[:-1])
        if (read_version != expected_version):
            print('Evaluation expects ' + expected_version +
                  ', but got dataset with ' + read_version,
                  file=sys.stderr)
        dataset = dataset_json['data']
    with open(prediction_file) as prediction_f:
        predictions = json.load(prediction_f)

    return evaluate(dataset, predictions)

In [14]:
logger = logging.getLogger(__name__)



def train(args, train_dataset, model, tokenizer):
    """ Train the model """
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(t_total * args.warmup_proportion), num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
            os.path.join(args.model_name_or_path, "scheduler.pt")
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Train batch size per GPU = %d", args.train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 1
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    mb = master_bar(range(int(args.num_train_epochs)))
    # Added here for reproductibility
    set_seed(args)

    for epoch in mb:
        epoch_iterator = progress_bar(train_dataloader, parent=mb)
        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            model.train()
            batch = tuple(t.to(args.device) for t in batch)

            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
                "start_positions": batch[3],
                "end_positions": batch[4],
            }

            if args.model_type in ["xlm", "roberta", "distilbert", "distilkobert", "xlm-roberta"]:
                del inputs["token_type_ids"]

            if args.model_type in ["xlnet", "xlm"]:
                inputs.update({"cls_index": batch[5], "p_mask": batch[6]})
                if args.version_2_with_negative:
                    inputs.update({"is_impossible": batch[7]})
                if hasattr(model, "config") and hasattr(model.config, "lang2id"):
                    inputs.update(
                        {"langs": (torch.ones(batch[0].shape, dtype=torch.int64) * args.lang_id).to(args.device)}
                    )

            outputs = model(**inputs)
            # model outputs are always tuple in transformers (see doc)
            loss = outputs[0]

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                # Log metrics
                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Only evaluate when single GPU otherwise metrics may not average well
                    if args.evaluate_during_training:
                        results = evaluation(args, model, tokenizer, global_step=global_step)
                        for key in sorted(results.keys()):
                            logger.info("  %s = %s", key, str(results[key]))

                    logging_loss = tr_loss

                # Save model checkpoint
                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    # Take care of distributed/parallel training
                    model_to_save = model.module if hasattr(model, "module") else model
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    if args.save_optimizer:
                        torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                        torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                        logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                break

        mb.write("Epoch {} done".format(epoch+1))

        if args.max_steps > 0 and global_step > args.max_steps:
            break

    return global_step, tr_loss / global_step


def evaluation(args, model, tokenizer, global_step=None):
    dataset, examples, features = load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True)

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(global_step))
    logger.info("  Num examples = %d", len(dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)

    all_results = []
    start_time = timeit.default_timer()

    for batch in progress_bar(eval_dataloader):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            if args.model_type in ["xlm", "roberta", "distilbert", "distilkobert", "xlm-roberta"]:
                del inputs["token_type_ids"]

            example_indices = batch[3]

            outputs = model(**inputs)
            
            start_logits = outputs[0] # bs 512
            end_logits = outputs[1] # torch.Tensor
            
            
            # print(len(start_logits)
            # print(len(start_logits[0]))
            
            
            
            
            
        # print(len(end_logits))
        # print(len(end_logits[0]))
        

        for i, example_index in enumerate(example_indices):
            start_logit = start_logits[i].cpu().tolist()
            end_logit = end_logits[i].cpu().tolist()
            # start_logits = start_logits[i]
            # end_logits = end_logits[i]
            # start_logits = start_logits[i]
            # end_logits = end_logits[i]
            eval_feature = features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            result = SquadResult(unique_id, start_logit, end_logit)

            all_results.append(result)

    evalTime = timeit.default_timer() - start_time
    logger.info("  Evaluation done in total %f secs (%f sec per example)", evalTime, evalTime / len(dataset))

    # Compute predictions
    output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(global_step))
    output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(global_step))

    if args.version_2_with_negative:
        output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(global_step))
    else:
        output_null_log_odds_file = None

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        args.n_best_size,
        args.max_answer_length,
        args.do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        args.verbose_logging,
        args.version_2_with_negative,
        args.null_score_diff_threshold,
        tokenizer,
    )

    # Compute the F1 and exact scores.
    results = squad_evaluate(examples, predictions)
    # Write the result
    # Write the evaluation result on file
    output_dir = os.path.join(args.output_dir, 'eval')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_eval_file = os.path.join(output_dir, "eval_result_{}_{}.txt".format(list(filter(None, args.model_name_or_path.split("/"))).pop(),
                                                                               global_step))

    with open(output_eval_file, "w", encoding='utf-8') as f:
        official_eval_results = eval_during_train(args, step=global_step)
        results.update(official_eval_results)

    return results


def load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False):
    # Load data features from cache or dataset file
    input_dir = args.data_dir if args.data_dir else "."
    cached_features_file = os.path.join(
        input_dir,
        "cached_{}_{}_{}".format(
            "dev" if evaluate else "train",
            list(filter(None, args.model_name_or_path.split("/"))).pop(),
            str(args.max_seq_length),
        ),
    )

    # Init features and dataset from cache if it exists
    if os.path.exists(cached_features_file):
        logger.info("Loading features from cached file %s", cached_features_file)
        features_and_dataset = torch.load(cached_features_file)
        features, dataset, examples = (
            features_and_dataset["features"],
            features_and_dataset["dataset"],
            features_and_dataset["examples"],
        )
    else:
        logger.info("Creating features from dataset file at %s", input_dir)

        if not args.data_dir and ((evaluate and not args.predict_file) or (not evaluate and not args.train_file)):
            try:
                import tensorflow_datasets as tfds
            except ImportError:
                raise ImportError("If not data_dir is specified, tensorflow_datasets needs to be installed.")

            if args.version_2_with_negative:
                logger.warn("tensorflow_datasets does not handle version 2 of SQuAD.")

            tfds_examples = tfds.load("squad")
            examples = SquadV1Processor().get_examples_from_dataset(tfds_examples, evaluate=evaluate)
        else:
            processor = SquadV2Processor() if args.version_2_with_negative else SquadV1Processor()
            if evaluate:
                examples = processor.get_dev_examples(os.path.join(args.data_dir),
                                                      filename=args.predict_file)
            else:
                examples = processor.get_train_examples(os.path.join(args.data_dir),
                                                        filename=args.train_file)

        features, dataset = squad_convert_examples_to_features(
            examples=examples,
            tokenizer=tokenizer,
            max_seq_length=args.max_seq_length,
            doc_stride=args.doc_stride,
            max_query_length=args.max_query_length,
            is_training=not evaluate,
            return_dataset="pt",
            threads=args.threads,
        )

        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save({"features": features, "dataset": dataset, "examples": examples}, cached_features_file)

    if output_examples:
        return dataset, examples, features
    return dataset


def main(args):
    # Read from config file and make args
    logger.info("Training/evaluation parameters {}".format(args))

    args.output_dir = os.path.join(args.output_dir)

    if args.doc_stride >= args.max_seq_length - args.max_query_length:
        logger.warning(
            "WARNING - You've set a doc stride which may be superior to the document length in some "
            "examples. This could result in errors when building features from the examples. Please reduce the doc "
            "stride or increase the maximum length to ensure the features are correctly built."
        )

    init_logger()
    set_seed(args)

    logging.getLogger("transformers.data.metrics.squad_metrics").setLevel(logging.WARN)  # Reduce model loading logs

    # Load pretrained model and tokenizer
    config = CONFIG_CLASSES[args.model_type].from_pretrained(
        args.model_name_or_path,
    )
    tokenizer = TOKENIZER_CLASSES[args.model_type].from_pretrained(
        args.model_name_or_path,
        do_lower_case=args.do_lower_case,
    )
    model = MODEL_FOR_QUESTION_ANSWERING[args.model_type].from_pretrained(
        args.model_name_or_path,
        config=config,
    )
    # GPU or CPU
    args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False)
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Evaluation - we can ask to evaluate all the checkpoints (sub-directories) in a directory
    results = {}
    if args.do_eval:
        checkpoints = list(
            os.path.dirname(c)
            for c in sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True))
        )
        if not args.eval_all_checkpoints:
            checkpoints = checkpoints[-1:]
        else:
            logging.getLogger("transformers.configuration_utils").setLevel(logging.WARN)  # Reduce model loading logs
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce model loading logs

        logger.info("Evaluate the following checkpoints: %s", checkpoints)

        for checkpoint in checkpoints:
            # Reload the model
            global_step = checkpoint.split("-")[-1]
            model = MODEL_FOR_QUESTION_ANSWERING[args.model_type].from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluation(args, model, tokenizer, global_step=global_step)
            result = dict((k + ("_{}".format(global_step) if global_step else ""), v) for k, v in result.items())
            results.update(result)

        output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as f_w:
            for key in sorted(results.keys()):
                f_w.write("{} = {}\n".format(key, str(results[key])))
                


## 2.3 Training

In [15]:
if __name__ == "__main__":

## 명령행 인터페이스, 사전 훈련된 모델을 로드하여 훈련하고 평가하는데까지 필요한 인자를 사용자 정의 인자로 설정하는 내용을 담고 있음.
## 사용법: python example.py --dataset_name squad.json .... 
## 주피터 환경을 위한 인자 설정법 
# "output_dir": "/content/gdrive/My Drive/Colab Notebooks/DFC615/koelectra-small-korquad-ckpt",

    os.chdir('/content/gdrive/MyDrive/Colab Notebooks/DFC615')
    gdrive_path = "/content/gdrive/My Drive/Colab Notebooks/DFC615"
    data_dir = gdrive_path
    output_dir = os.path.join(gdrive_path, "output")
    # train_file = os.path.join(gdrive_path, "KorQuAD_v1.0_train.json")
    # dev_file = os.path.join(gdrive_path, "KorQuAD_v1.0_dev.json")

    hyper_para={
    "task": "korquad",
    "data_dir": data_dir,
    "ckpt_dir": "ckpt",
    "train_file": "KorQuAD_v1.0_train.json",
    "predict_file": "KorQuAD_v1.0_dev.json",
    "threads": 4,
    "version_2_with_negative": False,
    "null_score_diff_threshold": 0.0,
    "max_seq_length": 512,
    "doc_stride": 128,
    "max_query_length": 64,
    "max_answer_length": 30,
    "n_best_size": 20,
    "verbose_logging": True,
    "overwrite_output_dir": True,
    "evaluate_during_training": True,
    "eval_all_checkpoints": True,
    "save_optimizer": False,
    "do_lower_case": False,
    "do_train": True,
    "do_eval": True,
    "num_train_epochs": 2,
    "weight_decay": 0.0,
    "gradient_accumulation_steps": 1,
    "adam_epsilon": 1e-8,
    "warmup_proportion": 0,
    "max_steps": -1,
    "max_grad_norm": 1.0,
    "no_cuda": False,
    "model_type": "koelectra-small-v3",
    "model_name_or_path": "monologg/koelectra-small-v3-discriminator",
    "output_dir": output_dir,
    "seed": 42,
    "train_batch_size": 32,
    "eval_batch_size": 64,
    "logging_steps": 500,
    "save_steps": 500,
    "learning_rate": 5e-5
    }

    main(AttrDict(hyper_para))

    # hyper_para['learning_rate']
    # hyper_para.learning_rate

06/25/2021 03:47:53 - INFO - filelock -   Lock 140583159158416 acquired on /root/.cache/torch/transformers/a95c1ddd5ce62adf8a4e5a75f70cb973b79c2d82705edc611de7b868ab917973.158ddd8c5effdf89c9e973250fe1db004e648ff1295bb63fd3abb0709141c558.lock


06/25/2021 03:47:53 - INFO - filelock -   Lock 140583159158416 released on /root/.cache/torch/transformers/a95c1ddd5ce62adf8a4e5a75f70cb973b79c2d82705edc611de7b868ab917973.158ddd8c5effdf89c9e973250fe1db004e648ff1295bb63fd3abb0709141c558.lock
06/25/2021 03:47:53 - INFO - filelock -   Lock 140581748799248 acquired on /root/.cache/torch/transformers/40bac8751309ca787dded2e47050b0ebb2b1292b04f5d7f6d8ff649c0164a99a.e060f8ca1da74aef5ab085f278fb3612042e7e4fc97a4917ed87540d3966ead6.lock


06/25/2021 03:47:53 - INFO - filelock -   Lock 140581748799248 released on /root/.cache/torch/transformers/40bac8751309ca787dded2e47050b0ebb2b1292b04f5d7f6d8ff649c0164a99a.e060f8ca1da74aef5ab085f278fb3612042e7e4fc97a4917ed87540d3966ead6.lock


06/25/2021 03:47:53 - INFO - filelock -   Lock 140581748797904 acquired on /root/.cache/torch/transformers/8e1af0a0f961eda7ae6a6a31d7b3a2b508ffba755d7ca857a0acd95ce4db0d8d.548bc4daa128698b4e44ee737f52215fad60b4eaa50f02442759fdc39cfbdb9a.lock


06/25/2021 03:47:53 - INFO - filelock -   Lock 140581748797904 released on /root/.cache/torch/transformers/8e1af0a0f961eda7ae6a6a31d7b3a2b508ffba755d7ca857a0acd95ce4db0d8d.548bc4daa128698b4e44ee737f52215fad60b4eaa50f02442759fdc39cfbdb9a.lock


06/25/2021 03:47:54 - INFO - filelock -   Lock 140583159159632 acquired on /root/.cache/torch/transformers/6e92e54c54ba31d8dbbb109128b5fc6b9904441c61e0eecf2b746dc118b92ce6.e907291bb60f0f7517ed4002a253cf54e79e3dd8181ba822b901b1c6f57a36ae.lock


06/25/2021 03:47:55 - INFO - filelock -   Lock 140583159159632 released on /root/.cache/torch/transformers/6e92e54c54ba31d8dbbb109128b5fc6b9904441c61e0eecf2b746dc118b92ce6.e907291bb60f0f7517ed4002a253cf54e79e3dd8181ba822b901b1c6f57a36ae.lock


Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['qa_outputs.weigh

KeyboardInterrupt: ignored

## 2.4 Prediction

In [64]:
test_file= os.path.join(gdrive_path,'test.json')
test_json = json.load(open(test_file,'r', encoding='utf-8'))

for sample in test_json['data']:
    context = sample['context']
    question = sample['question']
    print(f'context {context}')
    print(f'question {question}')    
    print('\n')
    
print(len(test_json['data']))


context 사회위해성이 있는 범죄라고 판단되는 행위에 대해서는 공안기관이 치안관리처벌법에 기초하여, 경고, 과료, 행정구류, 허가증의 취소, 외국인에 대하여는 국외추방이라고 하는 치안관리처벌을 부과한다. 치안관리처벌의 결정은 행정불복심사의 신청이나 행정소송에 의하여 다툴 수 있다. 그 외에, 일찍이 중국에는 법의 근거가 없는 행정처벌이나 법정의 절차를 준수하지 않는 행정처벌, 공포되지 않은 법령에 기초한 행정처벌을 볼 수 있었으나, 행정처벌법에는 이러한 행정처벌을 명문으로 금지하였다. 중국에는 일정한 직업을 가지지 아니한 자나 마약중독자 등에 대한 근로교정도 행해지고 있다. 근로교정의 시간은 최장 4년에 이르러, 그 절차나 운영에 관한 비판이 고조되고 있다.
question 사회에 해를 끼치는 범죄에 대해서 공안기관이 근거로 삼는 법안은 무엇인가?


context 이후 일본의 강요로 1943년 조선기독교단으로 강제 통합당한 감리교회는, 해방후 다시 분리되어 새로 교단을 구성하는 과정에서 재건파와 복흥파로 분리되는 등 홍역을 앓았다. 그러나 그 이후 다시 통합되어 1949년 현재의 이름이 채택되었다. 이후 1971년 경기연회 사건으로 총리원과 총회(갱신)측으로 분리되기도 하였으나[3], 이후 재통합돼 감리교단은 현재 대한민국 개신교 교파중에서 유일하게 교파 분열을 겪지 않은 교회이다. 다만 2008년 감독회장 선거로 시작된 기독교대한감리회 감독회장 선출 논쟁이 있었으나 2013년 현재 전용재 목사가 감독회장에 당선됨으로써 일단락됐다. 현재까지 감독 선출과 교회 운영은 감리회 본부와 총회를 중심으로 이루어지고 있다.
question 전용재 목사가 감독회장에 당선된 연도는?


context 일반적으로 자유가동을 하는 대부분의 유기체들은 기본적으로 유사한 위상 반응 곡선 (phase response curve, PRC)을 갖는다. 일주기 생체 리듬의 주기가 정확히 24시간이 아니기 때문에 빛이 주어지지 않은 상태에서는 위상의 변화가 일어나면서 그 다음 날의 활

In [70]:
tokenizer = ElectraTokenizer.from_pretrained("/content/gdrive/MyDrive/Colab Notebooks/DFC615/output/checkpoint-4000")
model = ElectraForQuestionAnswering.from_pretrained("/content/gdrive/MyDrive/Colab Notebooks/DFC615/output/checkpoint-4000")
prediction_qa = pipeline("question-answering", tokenizer=tokenizer, model=model)

context_question = {
    'question':'배터리 스와핑 시스템과 연관된 회사는?',

    'context':'"토요타 자동차 주식회사"는 일본 아이치현 도요타시에 본사를 둔 일본 굴지의 자동차 제조 회사이다.\
    "웨이모" LLC(Waymo LLC)는 구글 슬렉스의 연구소에서 개발하는 무인 자동차 기업이다. 구글 카라는 이름으로도 알려져 있다. 스탠포드대·카네기멜론대 연구팀, 무인자동차 경주인 그랜드 챌린지 우승자들을 영입해 무인자동차 사업을 시작하였다.\
    "현대 아이오닉"은 현대자동차의 첫 하이브리드 전용 차종이자 대한민국에서 최초로 선보인 하이브리드 전용 차종 및 EV 차종이다. 차명인 아이오닉은 전기적인 힘의 결합과 분리를 통해 새로운 에너지를 만들어내는 이온(ion)과 현대자동차의 독창성(unique)이 결합된\
    "테슬라"는 모든 수퍼차저 충전소마다 배터리 스와핑 시스템을 하나씩 설치할 것이라고 밝혔다. 하나를 설치하는데 50만 달러(6억원) 정도 비용이 든다. 기존 테슬라 전기차의 배터리를 통째로 빼내, 완충된 배터리로 교체해 주는 개념으로, 배터리 교체시간은 90초에 불과하다.'
}
bg_answer = prediction_qa(context_question)
print(bg_answer)
print(normalize_answer(bg_answer['answer']))

{'score': 0.6971803307533264, 'start': 352, 'end': 358, 'answer': '"테슬라"는'}
테슬라


## 2.5 기말고사 제출 리더보드용 파일생성

In [24]:
my_answer = {}
my_answer['id']=[]
my_answer['prediction_text']=[]

for idx, sample in enumerate(test_json['data']):
    context = sample['context']
    question = sample['question']
    answer_dict = prediction_qa({'question':question,'context': context})
    predict_text = normalize_answer(answer_dict["answer"])
    
    print(f'context {context}')
    print(f'question {question}')
    print(f'predict_text {predict_text}')
    print('\n')
    
    my_answer['id'].append(str(idx + 1))
    my_answer['prediction_text'].append(predict_text)
    



context 사회위해성이 있는 범죄라고 판단되는 행위에 대해서는 공안기관이 치안관리처벌법에 기초하여, 경고, 과료, 행정구류, 허가증의 취소, 외국인에 대하여는 국외추방이라고 하는 치안관리처벌을 부과한다. 치안관리처벌의 결정은 행정불복심사의 신청이나 행정소송에 의하여 다툴 수 있다. 그 외에, 일찍이 중국에는 법의 근거가 없는 행정처벌이나 법정의 절차를 준수하지 않는 행정처벌, 공포되지 않은 법령에 기초한 행정처벌을 볼 수 있었으나, 행정처벌법에는 이러한 행정처벌을 명문으로 금지하였다. 중국에는 일정한 직업을 가지지 아니한 자나 마약중독자 등에 대한 근로교정도 행해지고 있다. 근로교정의 시간은 최장 4년에 이르러, 그 절차나 운영에 관한 비판이 고조되고 있다.
question 사회에 해를 끼치는 범죄에 대해서 공안기관이 근거로 삼는 법안은 무엇인가?
predict_text 치안관리처벌법


context 이후 일본의 강요로 1943년 조선기독교단으로 강제 통합당한 감리교회는, 해방후 다시 분리되어 새로 교단을 구성하는 과정에서 재건파와 복흥파로 분리되는 등 홍역을 앓았다. 그러나 그 이후 다시 통합되어 1949년 현재의 이름이 채택되었다. 이후 1971년 경기연회 사건으로 총리원과 총회(갱신)측으로 분리되기도 하였으나[3], 이후 재통합돼 감리교단은 현재 대한민국 개신교 교파중에서 유일하게 교파 분열을 겪지 않은 교회이다. 다만 2008년 감독회장 선거로 시작된 기독교대한감리회 감독회장 선출 논쟁이 있었으나 2013년 현재 전용재 목사가 감독회장에 당선됨으로써 일단락됐다. 현재까지 감독 선출과 교회 운영은 감리회 본부와 총회를 중심으로 이루어지고 있다.
question 전용재 목사가 감독회장에 당선된 연도는?
predict_text 2013년


context 일반적으로 자유가동을 하는 대부분의 유기체들은 기본적으로 유사한 위상 반응 곡선 (phase response curve, PRC)을 갖는다. 일주기 생체 리듬의 주기가 정확히 24시간이 아니기 때문에 

In [ ]:
df = pd.DataFrame(my_answer)
df.head()

,id,prediction_text
0,1,치안관리처벌법
1,2,2013년
2,3,빛
3,4,타이베리움
4,5,11월 3일


In [ ]:
df.to_csv('my_answer.csv', index=False, encoding='utf-8')